In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=8b5dee4b51ae244ba904e705cdb831db56a8939e6262e59331b145845cb85259
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [14]:
from pyspark import SparkConf, SparkContext 
from google.colab import files
import sys
import os

def extractPairsOfProducts(transaction):

    products = list(transaction)

    returnedPairs = []
    
    for product1 in products:
        for product2 in products:
            if product1<product2:
                returnedPairs.append( ((product1, product2), 1) )
                
    return returnedPairs
    

if __name__=="__main__":
  conf = SparkConf().setAppName("Spark Lab06")
  sc = SparkContext.getOrCreate(conf=conf)

  #task 1
  lines = sc.textFile("ReviewsSample.csv")
  skipped = lines.filter(lambda line: line.startswith("Id,")==False)
  setKeyValLines = skipped.map(lambda s: (s.split(',')[2], s.split(',')[1])).distinct()
  userIdByKey = setKeyValLines.groupByKey()
  finalRDD = userIdByKey.mapValues(lambda l: list(l))

  print("UserID - List [ProductID]\n" + str(finalRDD.collect()) + "\n")


  #task 2
  pairsOfProductsOneRDD = finalRDD.values().flatMap(extractPairsOfProducts)
  setKeyValReducer = pairsOfProductsOneRDD.reduceByKey(lambda c1, c2: c1+c2)

  #task 3
  filteredKeyVal = setKeyValReducer.filter(lambda inTuple: inTuple[1]>1)
  sortedKeyVal = filteredKeyVal.sortBy(lambda inputTuple: inputTuple[1], False).cache()
  .saveAsTextFile(outputPath)

UserID - List [ProductID]
[('A2', ['B1', 'B3', 'B5']), ('A4', ['B1', 'B3', 'B4', 'B5']), ('A3', ['B3']), ('A1', ['B2']), ('A5', ['B5', 'B1', 'B3'])]

Frequencies of items pairs:
[(('B1', 'B3'), 3), (('B1', 'B5'), 3), (('B3', 'B5'), 3)]

